In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [38]:
movies_df=pd.read_excel("Bollywood_movie_imdb_ID.xlsx")

In [40]:
movies_df

,Movie Title,IMDB ID,title,Ratinng,Release Date,Genre,Cast,story,keywords,reviews,age,Review_keywords
0,3 Idiots,tt1187043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Taare Zameen Par,tt0986264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PK,tt2338151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dangal,tt5074352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rang De Basanti,tt0405508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3993,Mehmaan,tt8215918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3994,Sorry Daddy,tt36005951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3995,Raadha Aur Seeta,tt0378633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,Dafaa 302,tt0359129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
bolly_movies=movies_df.copy()

In [44]:
bolly_movies

,Movie Title,IMDB ID,title,Ratinng,Release Date,Genre,Cast,story,keywords,reviews,age,Review_keywords
0,3 Idiots,tt1187043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Taare Zameen Par,tt0986264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PK,tt2338151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dangal,tt5074352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rang De Basanti,tt0405508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3993,Mehmaan,tt8215918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3994,Sorry Daddy,tt36005951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3995,Raadha Aur Seeta,tt0378633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,Dafaa 302,tt0359129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
bolly_movies.dtypes

Movie Title         object
IMDB ID             object
title              float64
Ratinng            float64
Release Date       float64
Genre              float64
Cast               float64
story              float64
keywords           float64
reviews            float64
age                float64
Review_keywords    float64
dtype: object

In [48]:
bolly_movies.shape

(3998, 12)

In [50]:
bolly_movies.isna().sum()

Movie Title           0
IMDB ID               0
title              3998
Ratinng            3998
Release Date       3998
Genre              3998
Cast               3998
story              3998
keywords           3998
reviews            3998
age                3998
Review_keywords    3998
dtype: int64

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_movie_details(imdb_id):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        url = f"https://www.imdb.com/title/{imdb_id}/"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        movie = soup.find("main", class_="ipc-page-wrapper ipc-page-wrapper--base")

        movie_name = movie.find("span", class_="hero__primary-text").text
        movie_rating = movie.find("div", class_="sc-4dc495c1-0 fUqjJu").span.text
        movie_year = movie.find("ul", class_="ipc-inline-list ipc-inline-list--show-dividers sc-cb6a22b2-2 aFhKV baseAlt baseAlt").li.a.text

        # Duration
        duration = None
        details = soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-cb6a22b2-2 aFhKV baseAlt baseAlt')
        if details:
            items = details.find_all('li')
            for item in items:
                text = item.get_text(strip=True)
                if 'h' in text or 'm' in text:
                    duration = text
                    break

        # Genres
        genres = []
        movie_gen = movie.find_all("a", class_="ipc-chip ipc-chip--on-baseAlt")
        for g in movie_gen:
            genres.append(g.find("span", class_="ipc-chip__text").text)

        # Cast
        cast_list = []
        movie_cast = movie.find_all("div", class_="sc-10bde568-5 dWhYSc")
        for c in movie_cast:
            cast = c.find("div", class_="sc-10bde568-7 bhMzVl").a.text
            cast_list.append(cast)  

        return {
            "IMDB ID": imdb_id,
            "Movie Name": movie_name,
            "Rating": movie_rating,
            "Year": movie_year,
            "Duration": duration,
            "Genres": genres,
            "Cast": cast_list
        }

    except Exception as e:
        print(f"Error for {imdb_id}: {e}")
        return {
            "IMDB ID": imdb_id,
            "Movie Name": None,
            "Rating": None,
            "Year": None,
            "Duration": None,
            "Genres": [],
            "Cast": []
        }


In [54]:
# Assume your DataFrame is called df and has a column 'IMDB ID'
all_data = []

for imdb_id in bolly_movies['IMDB ID']:
    if pd.notnull(imdb_id): #notnull() is a function used to check whether a value is not missing (NaN).
        details = get_movie_details(imdb_id)
        all_data.append(details)
        time.sleep(1)  # Be nice to the server, avoid 429 rate limit

# Convert to DataFrame
Bolly_movies_det_df= pd.DataFrame(all_data)


Error for tt0105937: 'NoneType' object has no attribute 'text'
Error for tt24250998: 'NoneType' object has no attribute 'span'
Error for tt2085059: 'NoneType' object has no attribute 'text'
Error for tt23872886: 'NoneType' object has no attribute 'text'
Error for tt7493974: 'NoneType' object has no attribute 'text'
Error for tt0898266: 'NoneType' object has no attribute 'text'
Error for tt29868937: 'NoneType' object has no attribute 'span'
Error for tt19244304: 'NoneType' object has no attribute 'span'
Error for tt34569030: 'NoneType' object has no attribute 'text'
Error for tt30972834: 'NoneType' object has no attribute 'span'
Error for tt1833285: 'NoneType' object has no attribute 'text'
Error for tt33014566: 'NoneType' object has no attribute 'text'
Error for tt3561180: 'NoneType' object has no attribute 'text'
Error for tt27412560: 'NoneType' object has no attribute 'text'
Error for tt2660806: 'NoneType' object has no attribute 'text'
Error for tt37544113: 'NoneType' object has no 

In [56]:
Bolly_movies_det_df

,IMDB ID,Movie Name,Rating,Year,Duration,Genres,Cast
0,tt1187043,3 Idiots,8.4,2009,2h 50m,"[Buddy Comedy, Coming-of-Age, Quirky Comedy, S...","[Aamir Khan, Madhavan, Mona Singh, Sharman Jos..."
1,tt0986264,Taare Zameen Par,8.3,2007,2h 42m,"[Coming-of-Age, Drama, Family]","[Darsheel Safary, Aamir Khan, Tisca Chopra, Vi..."
2,tt2338151,PK,8.1,2014,2h 33m,"[Quest, Quirky Comedy, Satire, Comedy, Drama, ...","[Aamir Khan, Anushka Sharma, Sanjay Dutt, Boma..."
3,tt5074352,Dangal,8.3,2016,2h 41m,"[Coming-of-Age, Martial Arts, Action, Biograph...","[Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh..."
4,tt0405508,Rang De Basanti,8.1,2006,2h 47m,"[Political Drama, Tragedy, Comedy, Crime, Drama]","[Aamir Khan, Soha Ali Khan, Siddharth, Sharman..."
...,...,...,...,...,...,...,...
3993,tt8215918,Mehmaan,7.2,2018,20m,"[Drama, Short]","[Saheb Bhattacharjee, Kaushik Sen, Raima Sen]"
3994,tt36005951,None,None,None,None,[],[]
3995,tt0378633,Raadha Aur Seeta,6.3,1979,1h 49m,"[Drama, Family]","[Rajendranath Malhotra, Madan Puri, Leela Mish..."
3996,tt0359129,IPC Section 302,4.1,1975,1h 58m,"[Drama, Mystery]","[Randhir Kapoor, Rekha, Ajit Khan, Ashok Kumar..."


In [58]:
Bolly_movies_det_df.isna().sum()

IMDB ID         0
Movie Name    342
Rating        342
Year          342
Duration      768
Genres          0
Cast            0
dtype: int64

In [64]:
null_mov_df

,Movie Name
20,None
38,None
63,None
99,None
105,None
...,...
3987,Heera Moti
3989,Maine Jeena Seekh Liya
3990,Swarg Jaisaa Ghar
3992,None


In [136]:
Bolly_movies_det_df.loc[3987:4000]

,IMDB ID,Movie Name,Rating,Year,Duration,Genres,Cast
3987,tt0388995,Heera Moti,3.9,1960,None,[Drama],"[Shobha Khote, Ashim Kumar, Kumari Naaz, P. Ka..."
3988,tt0387692,Woh Jo Hasina,5.2,1983,2h 1m,[Romance],"[Mithun Chakraborty, Ranjeeta Kaur, Pran Sikan..."
3989,tt0374016,Maine Jeena Seekh Liya,2.8,1982,None,"[Drama, Romance]","[Mohammad Asif, Rajni Bala, Geeta Behl, Mohan ..."
3990,tt0364039,Swarg Jaisaa Ghar,2.9,1991,None,"[Drama, Family]","[Raj Babbar, Sumeet Saigal, Aashif Sheikh, Ani..."
3991,tt0351552,Phulwari,5.1,1984,2h 38m,"[Drama, Family]","[Debashree Roy, Shashi Puri, Ashok Saraf, Benj..."
3992,tt4897476,None,None,None,None,[],[]
3993,tt8215918,Mehmaan,7.2,2018,20m,"[Drama, Short]","[Saheb Bhattacharjee, Kaushik Sen, Raima Sen]"
3994,tt36005951,None,None,None,None,[],[]
3995,tt0378633,Raadha Aur Seeta,6.3,1979,1h 49m,"[Drama, Family]","[Rajendranath Malhotra, Madan Puri, Leela Mish..."
3996,tt0359129,IPC Section 302,4.1,1975,1h 58m,"[Drama, Mystery]","[Randhir Kapoor, Rekha, Ajit Khan, Ashok Kumar..."


In [68]:
Bolly_movies_det_df.to_excel("boly_movies_details.xlsx",index=False)

In [109]:
missing_details=Bolly_movies_det_df.loc[Bolly_movies_det_df["Movie Name"].isnull(),["IMDB ID","Movie Name","Rating","Year","Duration"]]
null_mov_name_ID_df=pd.DataFrame(data=missing_details)


In [99]:
nu_d=null_mov_dr_ID_df.copy()

In [181]:
null_mov_dr_ID_df.loc[890:2000]

,IMDB ID,Movie Name,Rating,Year,Duration
896,tt1844624,None,None,None,None
897,tt0098878,None,None,None,None
913,tt0157571,None,None,None,None
921,tt27793058,None,None,None,None
935,tt33484749,None,None,None,None
...,...,...,...,...,...
1953,tt1127870,Mumbai Salsa,5.1,2007,None
1976,tt0172163,Bambai Ka Babu,4.2,1996,None
1987,tt0358063,Raja Jani,6.5,1972,None
1991,tt32295747,None,None,None,None


In [111]:
null_mov_name_ID_df

,IMDB ID,Movie Name,Rating,Year,Duration
20,tt0105937,None,None,None,None
38,tt24250998,None,None,None,None
63,tt2085059,None,None,None,None
99,tt23872886,None,None,None,None
105,tt7493974,None,None,None,None
...,...,...,...,...,...
3947,tt30253363,None,None,None,None
3957,tt1755864,None,None,None,None
3986,tt35671338,None,None,None,None
3992,tt4897476,None,None,None,None


In [125]:
Bolly_movieee=Bolly_movies_det_df.copy()


In [127]:
for imdb_id in null_mov_dr_ID_df["IMDB ID"]:
    try:
        movie_data = get_movie_details(imdb_id)  # your scraping function
        if movie_data:
            # update the original DataFrame if needed
            Bolly_movieee.loc[Bolly_movieee["IMDB ID"] == imdb_id, ["Movie Name", "Rating", "Year", "Duration"]] = \
                movie_data["Movie Name"], movie_data["Rating"], movie_data["Year"], movie_data["Duration"]
    except Exception as e:
        print(f"Error for {imdb_id}: {e}")

Error for tt0105937: data not found
Error for tt24250998: data not found
Error for tt2085059: data not found
Error for tt23872886: data not found
Error for tt7493974: data not found
Error for tt0898266: data not found
Error for tt29868937: data not found
Error for tt19244304: data not found
Error for tt34569030: data not found
Error for tt30972834: data not found
Error for tt1833285: data not found
Error for tt33014566: data not found
Error for tt3561180: data not found
Error for tt27412560: data not found
Error for tt2660806: data not found
Error for tt37544113: data not found
Error for tt37500308: data not found
Error for tt28118211: data not found
Error for tt16098700: data not found
Error for tt12943690: data not found
Error for tt15282136: data not found
Error for tt35417736: data not found
Error for tt1595859: data not found
Error for tt10075030: data not found
Error for tt30037031: data not found
Error for tt0261214: data not found
Error for tt14993810: data not found
Error for 

In [131]:
Bolly_movieee.isna().sum()

IMDB ID         0
Movie Name    342
Rating        342
Year          342
Duration      768
Genres          0
Cast            0
dtype: int64

# without duration

In [185]:
without_duration=Bolly_movies_det_df.copy()

In [189]:
without_duration_details=without_duration.drop(labels="Duration",axis=1)

In [191]:
without_duration_details

,IMDB ID,Movie Name,Rating,Year,Genres,Cast
0,tt1187043,3 Idiots,8.4,2009,"[Buddy Comedy, Coming-of-Age, Quirky Comedy, S...","[Aamir Khan, Madhavan, Mona Singh, Sharman Jos..."
1,tt0986264,Taare Zameen Par,8.3,2007,"[Coming-of-Age, Drama, Family]","[Darsheel Safary, Aamir Khan, Tisca Chopra, Vi..."
2,tt2338151,PK,8.1,2014,"[Quest, Quirky Comedy, Satire, Comedy, Drama, ...","[Aamir Khan, Anushka Sharma, Sanjay Dutt, Boma..."
3,tt5074352,Dangal,8.3,2016,"[Coming-of-Age, Martial Arts, Action, Biograph...","[Aamir Khan, Sakshi Tanwar, Fatima Sana Shaikh..."
4,tt0405508,Rang De Basanti,8.1,2006,"[Political Drama, Tragedy, Comedy, Crime, Drama]","[Aamir Khan, Soha Ali Khan, Siddharth, Sharman..."
...,...,...,...,...,...,...
3993,tt8215918,Mehmaan,7.2,2018,"[Drama, Short]","[Saheb Bhattacharjee, Kaushik Sen, Raima Sen]"
3994,tt36005951,None,None,None,[],[]
3995,tt0378633,Raadha Aur Seeta,6.3,1979,"[Drama, Family]","[Rajendranath Malhotra, Madan Puri, Leela Mish..."
3996,tt0359129,IPC Section 302,4.1,1975,"[Drama, Mystery]","[Randhir Kapoor, Rekha, Ajit Khan, Ashok Kumar..."


In [193]:
without_duration_details.isna().sum()

IMDB ID         0
Movie Name    342
Rating        342
Year          342
Genres          0
Cast            0
dtype: int64

In [195]:
without_duration_details.to_excel("Bollywood_movies_without_duration_details.xlsx",index=False)